# Network Sniffing
- sniffing packets in unswitched network is fairly easy as packets are usually broadcasted to all the hosts in the network
- hub is a network device that broadcasts all the packets e.g. and simply running the interface card in promiscuous mode would collect all the data that was broadcasted
- sniffing packets in switched network is a bit challange
- if systems are behind router or network switch they're using swtiched network
    - packets are redirected to destination based on their MAC address

In [1]:
! ifconfig

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.231.152  netmask 255.255.255.0  broadcast 192.168.231.255
        inet6 fe80::20c:29ff:fed7:4d09  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:d7:4d:09  txqueuelen 1000  (Ethernet)
        RX packets 15469  bytes 19018045 (18.1 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 5759  bytes 379000 (370.1 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0
        device interrupt 19  base 0x2000  

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 3891  bytes 2469487 (2.3 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 3891  bytes 2469487 (2.3 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



In [2]:
#ifconfig <interfaceCard> promisc on
!ifconfig eth0 promisc on

on: Unknown host
ifconfig: `--help' gives usage information.


In [3]:
#ifconfig <interfaceCard>
! ifconfig eth0

eth0: flags=4419<UP,BROADCAST,RUNNING,PROMISC,MULTICAST>  mtu 1500
        inet 192.168.231.152  netmask 255.255.255.0  broadcast 192.168.231.255
        inet6 fe80::20c:29ff:fed7:4d09  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:d7:4d:09  txqueuelen 1000  (Ethernet)
        RX packets 15474  bytes 19018422 (18.1 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 5763  bytes 379252 (370.3 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0
        device interrupt 19  base 0x2000  



In [4]:
# install ftpd server for demo - remember FTP is not encrypted; typically use OpenSSH
# http://itsfoss.com/set-ftp-server-linux
! sudo -S apt install vsftpd -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
vsftpd is already the newest version (3.0.3-12).
0 upgraded, 0 newly installed, 0 to remove and 1611 not upgraded.


In [ ]:
# configure FTP Server
! man vsftpd.conf

### configure and restart FTP Server
- open /etc/vsftpd.conf to change settings
- allow users in /etc/passwd to login
    - local_enable=YES
- allow uploading to the FTP server
    - write_enable=YES
- bypass writable check; add the following directive
    - allow_writable_chroot=YES
- files are served by default from /srv/ftp
- restart FTP server
- add a new user say ftp-user

```
$ sudo adduser ftp-user
```

- make /home/ftp-user not writable

```
$ sudo chmod a-w /home/ftp-user
```

In [5]:
! echo user | sudo -S systemctl restart vsftpd

## install ftp client if missing

In [6]:
! echo user | sudo -S apt install ftp -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ftp is already the newest version (0.17-34.1+b1).
0 upgraded, 0 newly installed, 0 to remove and 1611 not upgraded.


In [ ]:
# run update if ftp install fails
! echo user | sudo -S apt update

In [7]:
# run tcpdump from a terminal
# tcpdump -l -X 'ip host <ip>'
! echo user | sudo tcpdump -l -X 'ip host 192.168.1.148'

tcpdump: verbose output suppressed, use -v or -vv for full protocol decode
listening on eth0, link-type EN10MB (Ethernet), capture size 262144 bytes
^C

0 packets captured
0 packets received by filter
0 packets dropped by kernel


In [ ]:
# run dsniff from a terminal
! dsniff -n

## Install and run telnet server
- not secure; use ssh
- https://www.cyberciti.biz/faq/howto-install-enable-telnet-on-debian-linux/
- not working!!

In [ ]:
! echo user | sudo -S apt install telnet telnetd -y

In [ ]:
! echo user | sudo -S /etc/init.d/xinetd restart  

# Active sniffing
- in a switched network environment, packets are only sent to the port they are destined for (using MAC addresses)
- promiscuous devices aren't able to sniff any additional packets that aren't destined for them
- a switch usually has multiple ethernet ports with its own MAC addresses

## arp spoofing

- arp spoofing allows the attacker's to redirect the victims packets to her system
- send a bunch of bogus ARP reply with attackers MAC but victim's IP; saying hey this is my IP and my MAC is this...
- victim then refreshes its ARP cache table (unless its explicitly marked as permanent) or adds new entry
- systems will accept ARP reply even if they don't send out ARP request
- also called ARP cache poisoning
- typically attackers would want to spoof gateway/router/switch that routes all the traffic from a local network out to the Internet

<img src="resources/arp-spoofing.png" />

## steps
    - we'll do a simple MitM between two victims for ethical reasons given we have permission from the victims

1. add the MAC addresses of the victims on the attackers machine

In [ ]:
# ping a victim ip
! ping -c 1 -w 1 <victim1 IP>

In [ ]:
# ping another vitim ip
! ping -c 1 -w 1 <victim2 IP>

In [ ]:
# test the arp cache
! arp -n

2. enable ip forwarding capability in the kernel so the victims do get their packets and are not suspicious of any spoofing activities

In [ ]:
# check ip_forward settings - 0 means disabled
! echo user | sudo -S cat /proc/sys/net/ipv4/ip_forward

In [ ]:
! echo user | sudo -S sysctl net.ipv4.ip_forward

In [ ]:
# enable if necessary on the fly
! echo user | sudo -S sysctl -w net.ipv4.ip_forward=1

### enable ip forwarding permanently
- open /etc/sysctl.conf file in an editor add the following line
```
net.ipv4.ip_forward = 1
```
- run the following command to enable the changes made

```
$ sysctl -p /etc/sysctl.conf
```

- or on Ubuntu run the following command

```
$ /etc/init.d/procps.sh restart
```

3. run arpspoof program to sniff packets on a switched LAN

In [ ]:
! man arpspoof

#### must run as root for arpspoof to work
```
$ sudo arpspoof -i <interface> -t <targetIP or victim1IP> -r <hostIP or victim2IP>
```

## Wireshark
- https://www.wireshark.org/docs/wsug_html_chunked/ChapterIntroduction.html
- https://www.wireshark.org/docs/wsug_html_chunked/ChWorkBuildDisplayFilterSection.html
- some filters - see manage saved bookmarks for exampes
- ip.src ==192.168.1. and ip.dst == 192.168.1.122